## Dampning Resistor

-  You input:
	Coil geometry (side length, conductor area) → estimates L and Rcoil
	Turn-off current I
	Period T
	Coil internal capacitance Ccoil
	MOSFET Coss (IXFH40N50Q2)
	Diode junction capacitance Cj (STTH3012)
- It computes:
	Stored inductive energy E=1/2 LI^2
	Average power P_avg=E/T
	Effective output capacitance:
	Stored inductive energy E=1/2 LI^2
	Average power P_avg=E/T
	
- Effective output capacitance:
	-  Ringing frequency (LC):

	f_0=1/(2π√(LC_tot ))

	-  Critical damping resistor for the parallel R across L||C (damping resistor across coil terminals):
	R_crit=1/2 √(L/C_tot )

	-  Suggested practical R_damprange: "  " 0.7R_critto 1.5R_crit
	-  “Energy that would initially charge the capacitance to V”:
	
    E_C (V)=1/2 C_tot V^2

	and the corresponding voltage if all inductive energy goes into C first:
    V_peak≈√(2E/C_tot )


In [17]:
import math
from dataclasses import dataclass
from typing import Optional


MU0 = 4e-7 * math.pi
RHO_CU = 1.68e-8  # Ohm·m (copper resistivity)


@dataclass
class CoilParams:
    side_m: float           # square loop side length [m]
    area_mm2: float         # copper cross section [mm^2]


@dataclass
class CapParams:
    c_coil_F: float         # coil internal capacitance [F]
    coss_mosfet_F: float    # MOSFET Coss [F] (IXFH40N50Q2 at relevant Vds)
    cj_diode_F: float       # diode junction capacitance [F] (STTH3012 at relevant Vr)


@dataclass
class Inputs:
    coil: CoilParams
    caps: CapParams
    I_turnoff_A: float      # current at turn-off [A]
    period_s: float         # repetition period [s]


@dataclass
class Results:
    R_coil_ohm: float
    L_H: float

    E_L_J: float
    P_avg_W: float

    C_out_F: float
    C_tot_F: float

    f0_Hz: float
    Rcrit_ohm: float
    Rdamp_suggest_low_ohm: float
    Rdamp_suggest_high_ohm: float

    Vpeak_if_all_energy_to_C_V: float


def estimate_coil_R_L_square_loop(side_m: float, area_mm2: float) -> tuple[float, float]:
    """
    Estimate coil DC resistance and a crude square-loop inductance.

    R = rho * length / area
    L approximation: 2*mu0*a*(ln(1.08*a/r_wire) + 0.25)
    """
    length = 4.0 * side_m
    A = area_mm2 * 1e-6  # mm^2 -> m^2
    R = RHO_CU * length / A

    r_wire = math.sqrt(A / math.pi)
    a = side_m
    L = 2.0 * MU0 * a * (math.log(1.08 * a / r_wire) + 0.25)
    return R, L


def compute_tem_damping(inputs: Inputs) -> Results:
    # coil
    R_coil, L = estimate_coil_R_L_square_loop(inputs.coil.side_m, inputs.coil.area_mm2)

    # energy per event + average power
    I = inputs.I_turnoff_A
    T = inputs.period_s
    E_L = 0.5 * L * I * I
    P_avg = E_L / T

    # effective output capacitance (your definition)
    C_out = 0.5 * inputs.caps.coss_mosfet_F + 0.5 * inputs.caps.cj_diode_F
    C_tot = inputs.caps.c_coil_F + C_out

    if C_tot <= 0:
        raise ValueError("C_tot must be > 0")

    # resonant frequency of L with total capacitance
    f0 = 1.0 / (2.0 * math.pi * math.sqrt(L * C_tot))

    # For parallel R across L||C:
    # equation: v'' + (1/(R*C)) v' + (1/(L*C)) v = 0
    # critical damping: 1/(R*C) = 2*sqrt(1/(L*C))
    # => Rcrit = 0.5 * sqrt(L/C)
    Rcrit = 0.5 * math.sqrt(L / C_tot)

    # Suggest practical range around critical damping
    Rd_low = 0.98 * Rcrit
    Rd_high = 1.02 * Rcrit

    # If all inductive energy initially charges C_tot:
    Vpeak = math.sqrt(2.0 * E_L / C_tot)

    return Results(
        R_coil_ohm=R_coil,
        L_H=L,
        E_L_J=E_L,
        P_avg_W=P_avg,
        C_out_F=C_out,
        C_tot_F=C_tot,
        f0_Hz=f0,
        Rcrit_ohm=Rcrit,
        Rdamp_suggest_low_ohm=Rd_low,
        Rdamp_suggest_high_ohm=Rd_high,
        Vpeak_if_all_energy_to_C_V=Vpeak,
    )


def fmt_si(x: float, unit: str) -> str:
    # simple SI formatter
    scales = [
        (1e-12, "p"),
        (1e-9, "n"),
        (1e-6, "u"),
        (1e-3, "m"),
        (1.0, ""),
        (1e3, "k"),
        (1e6, "M"),
    ]
    ax = abs(x)
    best = (1.0, "")
    for s, p in scales:
        if ax >= s:
            best = (s, p)
    s, p = best
    return f"{x/s:.3g} {p}{unit}"


if __name__ == "__main__":
    # ---- Example values (REPLACE with datasheet values at your operating voltage) ----
    # IMPORTANT:
    # Coss and diode capacitance are strong functions of voltage.
    # Use the value at the relevant Vds/Vr (e.g. 400–500 V), not the low-voltage value.

    inp = Inputs(
        coil=CoilParams(side_m=40.0, area_mm2=4.0),
        caps=CapParams(
            c_coil_F=200e-12,         # 200 pF coil self-capacitance (example)
            coss_mosfet_F=300e-12,    # IXFH40N50Q2 Coss at operating V (example)
            cj_diode_F=100e-12,       # STTH3012 junction C at operating V (example)
        ),
        I_turnoff_A=300.0,
        period_s=0.02,
    )

    r = compute_tem_damping(inp)

    print("---------------------------------------------------")
    print("|      TEM Damping / Cout / Ccoil Calculator      |")
    print("---------------------------------------------------")
    print(f"R_coil          : {r.R_coil_ohm:.4f} Ω")
    print(f"L               : {fmt_si(r.L_H, 'H')}")

    print(f"E_L per event   : {r.E_L_J:.3f} J")
    # print(f"P_avg (repeat)  : {r.P_avg_W:.1f} W   (if dissipated each period)")

    print(f"C_out           : {fmt_si(r.C_out_F, 'F')}   (0.5*Coss + 0.5*Cdiode)")
    print(f"C_tot           : {fmt_si(r.C_tot_F, 'F')}   (Ccoil + Cout)")

    print(f"f0 (L//C_tot)   : {fmt_si(r.f0_Hz, 'Hz')}")
    print(f"R_crit (parallel): {r.Rcrit_ohm:.2f} Ω")
    print(f"Suggested Rdamp : {r.Rdamp_suggest_low_ohm:.1f} … {r.Rdamp_suggest_high_ohm:.1f} Ω")
    print("---------------------------------------------------")
    # print(f"Vpeak (if E->C) : {r.Vpeak_if_all_energy_to_C_V:.0f} V") # without avalanche diode
    print()
    print("Notes:")
    print("- Update Coss and diode Cj with datasheet values at your actual operating voltage.")
    print("- R_crit is for a parallel damping resistor across the coil terminals (L||C).")


---------------------------------------------------
|      TEM Damping / Cout / Ccoil Calculator      |
---------------------------------------------------
R_coil          : 0.6720 Ω
L               : 1.09 mH
E_L per event   : 48.871 J
C_out           : 200 pF   (0.5*Coss + 0.5*Cdiode)
C_tot           : 400 pF   (Ccoil + Cout)
f0 (L//C_tot)   : 241 kHz
R_crit (parallel): 823.87 Ω
Suggested Rdamp : 807.4 … 840.3 Ω
---------------------------------------------------

Notes:
- Update Coss and diode Cj with datasheet values at your actual operating voltage.
- R_crit is for a parallel damping resistor across the coil terminals (L||C).
